# Nearest Neighbor price based Collaborative Filtering

In [155]:
import pandas as pd
import numpy as np

In [156]:
combined= pd.read_csv('../data/semi_all_merged_dist.csv')

In [157]:
# 10개의 대분류로 카테고리 매칭
category_mapping = {
    # 1. 전자제품/기술
    'PC': '전자제품/기술',
    'PC 게이머': '전자제품/기술',
    '가전제품': '전자제품/기술',
    '가전제품 2': '전자제품/기술',
    '소형 가전제품': '전자제품/기술',
    '유선 전화': '전자제품/기술',
    '전화기': '전자제품/기술',
    '컴퓨터/액세서리': '전자제품/기술',
    '태블릿/프린터/이미징': '전자제품/기술',
    '오디오': '전자제품/기술',
    '전자제품': '전자제품/기술',
    '휴대용 가전/오븐/커피머신': '전자제품/기술',

    # 2. 가구/인테리어
    '가구/인테리어': '가구/인테리어',
    '부엌/서비스/다이닝/정원 가구': '가구/인테리어',
    '거실 가구': '가구/인테리어',
    '침대/매트리스/소파': '가구/인테리어',
    '침실 가구': '가구/인테리어',
    '사무용 가구': '가구/인테리어',
    '정원용품': '가구/인테리어',

    # 3. 패션/액세서리
    '패션/가방/액세서리': '패션/액세서리',
    '가방/액세서리': '패션/액세서리',
    '패션/남성복': '패션/액세서리',
    '패션/여성복': '패션/액세서리',
    '패션/아동복': '패션/액세서리',
    '패션/신발': '패션/액세서리',
    '패션/스포츠': '패션/액세서리',
    '패션/속옷 및 비치웨어': '패션/액세서리',
    '시계/선물': '패션/액세서리',

    # 4. 뷰티/건강
    '뷰티/건강': '뷰티/건강',
    '향수/미용': '뷰티/건강',
    '기저귀/위생용품': '뷰티/건강',

    # 5. 식품/음료
    '식품': '식품/음료',
    '식품/음료': '식품/음료',
    '음료': '식품/음료',

    # 6. 미디어/엔터테인먼트
    'DVD/블루레이': '미디어/엔터테인먼트',
    '콘솔/게임': '미디어/엔터테인먼트',
    '음악 CD/DVD': '미디어/엔터테인먼트',
    '음악': '미디어/엔터테인먼트',
    '영화/사진': '미디어/엔터테인먼트',

    # 7. 도서/교육
    '기술 도서': '도서/교육',
    '일반 도서': '도서/교육',
    '수입 도서': '도서/교육',

    # 8. 스포츠/야외 활동
    '스포츠/레저': '스포츠/야외 활동',
    '패션/스포츠': '스포츠/야외 활동',

    # 9. 유아/애완용품
    '유아용품': '유아/애완용품',
    '장난감': '유아/애완용품',
    '애완동물 용품': '유아/애완용품',

    # 10. 기타
    '기타': '기타',
    '크리스마스 용품': '기타',
    '파티용품': '기타',
    '표지판/보안': '기타',
    '독특한 물품': '기타',
    '라쿠진(요리 관련)': '기타',
    '보험/서비스': '기타',
    '마켓플레이스': '기타',
    '농업/산업/상업': '기타',
    '산업/상업/비즈니스': '기타',
    '건축/공구/건설': '기타',
    '건축/공구/도구': '기타',
    '건축/공구/보안': '기타',
    '건축/공구/정원': '기타',
    '건축/공구/조명': '기타',
    '주택/건축': '기타',
}



In [158]:
combined['product_category_name_english'] = combined['product_category_name'].map(category_mapping)

In [159]:
combined.head()

,Unnamed: 0,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,seller_state,seller_lat,seller_lng,customer_city,customer_state,customer_lat,customer_lng,order_delivered_customer_time,dist,product_category_name_english
0,0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,1.0,4244733e06e7ecb4970a6e2683c13e61,...,SP,-22.498183,-44.123614,campos dos goytacazes,RJ,-21.758076,-41.312633,104972,301.005664,기타
1,1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,1.0,e5f2d52b802189ee658865ca93d83a8f,...,SP,-23.566258,-46.518417,santa fe do sul,SP,-20.212393,-50.941471,696564,589.274140,유아/애완용품
2,2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,1.0,c777355d18b72b67abbeef9df44fd0fd,...,MG,-22.264094,-46.158564,para de minas,MG,-19.860439,-44.597972,520948,312.495046,가구/인테리어
3,3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,1.0,7634da152a4610f1595efa32f14722fc,...,SP,-20.548228,-47.395897,atibaia,SP,-23.144923,-46.539830,345879,301.951753,뷰티/건강
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,1.0,ac6c3623068f30de03045865e4e10089,...,PR,-22.931427,-53.133759,varzea paulista,SP,-23.249008,-46.824961,1148182,646.221788,가구/인테리어


In [160]:
review = pd.read_csv('../data/olist_order_reviews_dataset.csv')[['order_id','review_score']]

In [161]:
review

,order_id,review_score
0,73fc7af87114b39712e6da79b0a377eb,4
1,a548910a1c6147796b98fdf73dbeba33,5
2,f9e4b658b201a9f2ecdecbb34bed034b,5
3,658677c97b385a9be170737859d3511b,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,5
...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,5
99220,22ec9f0669f784db00fa86d035cf8602,5
99221,55d4004744368f5571d1f590031933e4,5
99222,7725825d039fc1f0ceb7635e3f7d9206,4


In [162]:
combined = pd.merge(combined, review, on='order_id', how='left')

In [163]:
combined.rename(columns={'customer_id': 'customer_unique_id'}, inplace=True)

In [164]:
combined

,Unnamed: 0,order_id,customer_unique_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,seller_lat,seller_lng,customer_city,customer_state,customer_lat,customer_lng,order_delivered_customer_time,dist,product_category_name_english,review_score
0,0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,1.0,4244733e06e7ecb4970a6e2683c13e61,...,-22.498183,-44.123614,campos dos goytacazes,RJ,-21.758076,-41.312633,104972,301.005664,기타,5.0
1,1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,1.0,e5f2d52b802189ee658865ca93d83a8f,...,-23.566258,-46.518417,santa fe do sul,SP,-20.212393,-50.941471,696564,589.274140,유아/애완용품,4.0
2,2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,1.0,c777355d18b72b67abbeef9df44fd0fd,...,-22.264094,-46.158564,para de minas,MG,-19.860439,-44.597972,520948,312.495046,가구/인테리어,5.0
3,3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,1.0,7634da152a4610f1595efa32f14722fc,...,-20.548228,-47.395897,atibaia,SP,-23.144923,-46.539830,345879,301.951753,뷰티/건강,4.0
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,1.0,ac6c3623068f30de03045865e4e10089,...,-22.931427,-53.133759,varzea paulista,SP,-23.249008,-46.824961,1148182,646.221788,가구/인테리어,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110248,109614,fffc94f6ce00a00581880bf54a75a037,b51593916b4b8e0d6f66f2ae24f2673d,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,1.0,4aa6014eceb682077f9dc4bffebc05b0,...,-26.912429,-48.677381,sao luis,MA,-2.490420,-44.303287,1334860,2755.314485,NaN,5.0
110249,109615,fffcd46ef2263f404302a634eb57f7eb,84c5d4fbaf120aae381fad077416eaa0,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,1.0,32e07fd915822b0765e448c4dd74c828,...,-23.535536,-46.643052,curitiba,PR,-25.601485,-49.317700,563215,354.848852,전자제품/기술,5.0
110250,109616,fffce4705a9662cd70adb13d4a31832d,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,1.0,72a30483855e2eafc67aee5dc2560482,...,-25.469214,-49.293935,sao paulo,SP,-23.597813,-46.650272,162548,338.827218,스포츠/야외 활동,5.0
110251,109617,fffe18544ffabc95dfada21779c9644f,b5e6afd5a41800fdf401e0272ca74655,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,1.0,9c422a519119dcad7575db5af1ba540e,...,-23.636657,-46.694780,vinhedo,SP,-23.036935,-46.982526,97007,72.870476,전자제품/기술,5.0


In [165]:
combined.dropna(subset=['product_category_name_english'], inplace=True)

In [166]:
combined.to_csv('combined.csv', index=False)

In [167]:
df1= combined[['product_id','customer_unique_id','product_category_name_english','review_score','price']]

In [168]:
df1['product_category_name_english'].unique()

array(['기타', '유아/애완용품', '가구/인테리어', '뷰티/건강', '전자제품/기술', '도서/교육', '패션/액세서리',
       '스포츠/야외 활동', '미디어/엔터테인먼트', '식품/음료'], dtype=object)

In [169]:
df1.head()

,product_id,customer_unique_id,product_category_name_english,review_score,price
0,4244733e06e7ecb4970a6e2683c13e61,3ce436f183e68e07877b285a838db11a,기타,5.0,58.90
1,e5f2d52b802189ee658865ca93d83a8f,f6dd3ec061db4e3987629fe6b26e5cce,유아/애완용품,4.0,239.90
2,c777355d18b72b67abbeef9df44fd0fd,6489ae5e4333f3693df5ad4372dab6d3,가구/인테리어,5.0,199.00
3,7634da152a4610f1595efa32f14722fc,d4eb9395c8c0431ee92fce09860c5a06,뷰티/건강,4.0,12.99
4,ac6c3623068f30de03045865e4e10089,58dbd0b2d70206bf40e62cd34e84d795,가구/인테리어,5.0,199.90


In [170]:
df1.groupby('product_category_name_english')['price'].mean().sort_values(ascending=False).head()

product_category_name_english
패션/액세서리       157.695363
기타            148.341614
미디어/엔터테인먼트    130.994331
뷰티/건강         126.377074
유아/애완용품       120.935943
Name: price, dtype: float64

In [171]:
price = pd.DataFrame(df1.groupby('product_category_name_english')['price'].mean())
price.head()

,price
product_category_name_english,
가구/인테리어,105.781837
기타,148.341614
도서/교육,79.831027
미디어/엔터테인먼트,130.994331
뷰티/건강,126.377074


# create a Pivot matrix

In [172]:
df2 = df1.pivot_table(index='product_category_name_english',columns='customer_unique_id',values='price').fillna(0)
df2.head()

customer_unique_id,00012a2ce6f8dcda20d059ce98491703,000161a058600d5901f007fab4c27140,0001fd6190edaaf884bcaf3d49edf079,0002414f95344307404f0ace7a26f1d5,0004164d20a9e969af783496f3408652,000419c5494106c306a97b5635748086,00046a560d407e99b969756e0b10f282,00050bf6e01e69d5c0fd612f1bcfb69c,000598caf2ef4117407665ac33275130,00062b33cb9f6fe976afdcff967ea74d,...,fff5169e583fd07fac9fec88962f189d,fff55ba8dffa552b6fdfd86c2e806459,fff5dd22d522cf28a902185817642a2e,fff675a0d5924b9162b4a1bf410466cd,fff7466a253c0e59499ea943462c10f9,fff93c1da78dafaaa304ff032abc6205,fffa0238b217e18a8adeeda0669923a3,fffcb937e9dd47a13f05ecb8290f4d3e,fffeda5b6d849fbd39689bb92087f431,ffffa3172527f765de70084a7e53aae8
product_category_name_english,,,,,,,,,,,,,,,,,,,,,
가구/인테리어,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,10.9
기타,0.0,0.0,0.0,149.9,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
도서/교육,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,66.296667,0.0,78.0,0.0,0.0
미디어/엔터테인먼트,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
뷰티/건강,0.0,54.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.99,...,144.0,14.9,0.0,0.0,152.99,0.000000,35.0,0.0,0.0,0.0


In [173]:
from scipy.sparse import csr_matrix

price_df_matrix = csr_matrix(df2.values)

In [174]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(price_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [175]:
price_df_matrix.shape

(10, 73788)

# choose a random product

In [176]:
query_index = np.random.choice(df2.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(df2.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

7


# Based on that product what we have choosen 5 nearest distance product get recommended based on prices

In [177]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(df2.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, df2.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 유아/애완용품:

1: 기타, with distance of 0.9984374498428045:
2: 가구/인테리어, with distance of 0.9997223086229795:
3: 뷰티/건강, with distance of 0.9999025975356173:
4: 스포츠/야외 활동, with distance of 0.9999227356330371:
5: 패션/액세서리, with distance of 0.9999423715656724:


# Calculate the accuracy

In [178]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
  

In [179]:
df3=pd.read_csv("combined.csv",usecols=['product_category_name_english','price'])
df3

,price,product_category_name_english
0,58.90,기타
1,239.90,유아/애완용품
2,199.00,가구/인테리어
3,12.99,뷰티/건강
4,199.90,가구/인테리어
...,...,...
84091,55.00,패션/액세서리
84092,119.85,기타
84093,350.00,전자제품/기술
84094,99.90,스포츠/야외 활동


In [180]:
df4 = pd.get_dummies(df3)
df4

,price,product_category_name_english_가구/인테리어,product_category_name_english_기타,product_category_name_english_도서/교육,product_category_name_english_미디어/엔터테인먼트,product_category_name_english_뷰티/건강,product_category_name_english_스포츠/야외 활동,product_category_name_english_식품/음료,product_category_name_english_유아/애완용품,product_category_name_english_전자제품/기술,product_category_name_english_패션/액세서리
0,58.90,False,True,False,False,False,False,False,False,False,False
1,239.90,False,False,False,False,False,False,False,True,False,False
2,199.00,True,False,False,False,False,False,False,False,False,False
3,12.99,False,False,False,False,True,False,False,False,False,False
4,199.90,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
84091,55.00,False,False,False,False,False,False,False,False,False,True
84092,119.85,False,True,False,False,False,False,False,False,False,False
84093,350.00,False,False,False,False,False,False,False,False,True,False
84094,99.90,False,False,False,False,False,True,False,False,False,False


In [181]:
X = df4.drop('price',axis=1)
y= df4['price']

In [182]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,random_state=0)

In [183]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

lreg = LinearRegression()
lreg.fit(X_train, y_train)

LinearRegression()

In [184]:
pred_train = lreg.predict(X_train)
train_score = np.sqrt(mean_squared_log_error(y_train,pred_train))

In [185]:
pred_test = lreg.predict(X_test)
test_score = np.sqrt(mean_squared_log_error(y_test,pred_test))

In [186]:
print('Training score:', train_score)
print('Test score:', test_score)

Training score: 1.0338148281221762
Test score: 1.028493632457869
